# 動物エキスパートシステムの実装

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners)からの例です。

このサンプルでは、いくつかの物理的特徴に基づいて動物を判断する単純な知識ベースシステムを実装します。システムは以下のようなAND-ORツリーで表現できます（これは全体のツリーの一部で、簡単にルールを追加できます）:

![](../../../../../../translated_images/ja/AND-OR-Tree.5592d2c70187f283.webp)


## 自作の後退推論を持つエキスパートシステムシェル

生産規則に基づいた知識表現のための簡単な言語を定義してみましょう。ルールを定義するキーワードとしてPythonクラスを使用します。基本的に3種類のクラスがあります：
* `Ask` はユーザーに質問する必要がある質問を表します。可能な回答のセットを含んでいます。
* `If` はルールを表し、ルールの内容を格納するための構文糖衣です。
* `AND`/`OR` はツリーのAND/ORの枝を表すクラスです。引数のリストを内部に格納します。コードを簡素化するために、すべての機能は親クラス `Content` に定義されています。


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

私たちのシステムでは、作業記憶は**属性-値のペア**としての**事実**のリストを含みます。知識ベースは、行動（作業記憶に挿入されるべき新しい事実）を条件にマッピングする大きな辞書として定義できます。条件はAND-OR式で表現されます。また、一部の事実は`Ask`することができます。


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

逆推論を行うために、`Knowledgebase` クラスを定義します。これには以下が含まれます：
* 動作中の `memory` - 属性を値にマッピングする辞書
* 先に定義した形式の Knowledgebase の `rules`

主なメソッドは以下の2つです：
* `get` は属性の値を取得し、必要に応じて推論を行います。例えば、`get('color')` は色のスロットの値を取得します（必要に応じて問い合わせを行い、後で使用するために作業メモリに値を保存します）。`get('color:blue')` と指定した場合は色を問い合わせ、その色に応じて「y」または「n」の値を返します。
* `eval` は実際の推論を行います。つまり、AND/ORツリーを辿り、サブゴールを評価するなどの処理を行います。


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

さて、動物の知識ベースを定義し、相談を行いましょう。この呼び出しでは質問が行われます。はい・いいえの質問には `y`/`n` と入力して答え、複数選択肢の質問には番号（0..N）を指定して答えてください。


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Expertaを使った順方向推論

次の例では、知識表現用のライブラリの一つである[Experta](https://github.com/nilp0inter/experta)を使って順方向推論を実装してみます。**Experta**は、Pythonで順方向推論システムを作成するためのライブラリで、古典的なシステムである[CLIPS](http://www.clipsrules.net/index.html)に似た設計がされています。

順方向連鎖を自分で実装することも多くの問題なく可能ですが、素朴な実装は通常効率的ではありません。より効果的なルールマッチングには、特別なアルゴリズムである[Rete](https://en.wikipedia.org/wiki/Rete_algorithm)が使われます。


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

私たちはシステムを `KnowledgeEngine` をサブクラス化したクラスとして定義します。各ルールは `@Rule` アノテーションが付いた別々の関数で定義されており、このアノテーションはルールがいつ発動すべきかを指定します。ルール内では `declare` 関数を使って新しいファクトを追加でき、これらのファクトを追加すると順方向推論エンジンによってさらにいくつかのルールが呼び出されます。


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

ナレッジベースを定義したら、いくつかの初期事実でワーキングメモリを埋め、次に推論を実行するために `run()` メソッドを呼び出します。結果として、新しく推論された事実がワーキングメモリに追加されるのがわかります。これには、動物に関する最終事実も含まれます（すべての初期事実を正しく設定した場合）。


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性を期しておりますが、自動翻訳には誤りや不正確な表現が含まれる可能性があります。原文の言語によるオリジナル文書が正式な情報源とみなされます。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用によって生じたいかなる誤解や解釈の違いについても、一切責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
